In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from aana.api.sdk import AanaSDK
from aana.api.sdk import get_deployment


aana_sdk = AanaSDK(port=8000)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-10 14:38:17,654	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-10 14:38:21,704	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-04-10 14:38:22,894	INFO worker.py:1715 -- Sta

(ServeReplica:whisper_deployment_medium:WhisperDeployment pid=106059) INFO 2024-04-10 14:58:15,791 whisper_deployment_medium_WhisperDeployment xpUONH e7d3e39a-8c4f-48bc-92fd-f2a94e801db1 /transcribe_video_stream replica.py:772 - TRANSCRIBE_STREAM OK 9334.5ms
(ServeReplica:RequestHandler:RequestHandler pid=108862) INFO 2024-04-10 14:58:15,799 RequestHandler_RequestHandler FyvJrc e7d3e39a-8c4f-48bc-92fd-f2a94e801db1 /transcribe_video_stream replica.py:772 - __CALL__ OK 14691.8ms


In [3]:
from typing import TypedDict
from aana.models.pydantic.video_input import VideoInput
from aana.utils.video import download_video
from aana.models.core.video import Video


class VideoInfo(TypedDict):
    title: str
    description: str


async def get_video_info(video_input: VideoInput) -> VideoInfo:
    video: Video = download_video(video_input=video_input)
    return {
        "title": video.title,
        "description": video.description,
    }


aana_sdk.register_endpoint(
    name="get_video_info",
    path="/get_video_info",
    summary="Get video info",
    func=get_video_info,
)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [4]:
aana_sdk.start_request_handler()

(ProxyActor pid=105826) INFO 2024-04-10 14:38:34,432 proxy 172.17.0.2 proxy.py:1143 - Proxy actor bb16db390f0a18f7378a74c401000000 starting on node 76d38b184291dfb1fdfcad111f3b473bcf040eba37604aa53e56252c.
(ProxyActor pid=105826) INFO 2024-04-10 14:38:34,439 proxy 172.17.0.2 proxy.py:1357 - Starting HTTP server on node: 76d38b184291dfb1fdfcad111f3b473bcf040eba37604aa53e56252c listening on port 8000
(ServeController pid=105747) INFO 2024-04-10 14:38:34,545 controller 105747 deployment_state.py:1547 - Deploying new version of deployment RequestHandler in application 'RequestHandler'. Setting initial target number of replicas to 1.
(ProxyActor pid=105826) INFO:     Started server process [105826]
(ServeController pid=105747) INFO 2024-04-10 14:38:34,649 controller 105747 deployment_state.py:1831 - Adding 1 replica to deployment RequestHandler in application 'RequestHandler'.
(ServeReplica:RequestHandler:RequestHandler pid=105903) /root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/

2024-04-10 14:38:42,599	INFO router.py:959 -- Using router <class 'ray.serve._private.router.PowerOfTwoChoicesReplicaScheduler'>.
2024-04-10 14:38:42,626	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': {'RequestHandler#RequestHandler#xeWzgo'}.


DeploymentHandle(deployment='RequestHandler')

In [5]:
import requests, json

data = {"video_input": {"url": "https://www.youtube.com/watch?v=9bZkp7q19f0"}}
requests.post(
    "http://127.0.0.1:8000/get_video_info", data={"body": json.dumps(data)}
).json()

{'detail': 'Not Found'}

In [6]:
from aana.deployments.whisper_deployment import (
    WhisperComputeType,
    WhisperConfig,
    WhisperDeployment,
    WhisperModelSize,
)


whisper_deployment_medium = WhisperDeployment.options(
    num_replicas=1,
    max_concurrent_queries=1000,
    ray_actor_options={"num_gpus": 0.25},
    user_config=WhisperConfig(
        model_size=WhisperModelSize.MEDIUM,
        compute_type=WhisperComputeType.FLOAT16,
    ).model_dump(),
)

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_size" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
aana_sdk.register_deployment(
    "whisper_deployment_medium",
    whisper_deployment_medium,
)

In [8]:
from typing import TypedDict
from aana.models.pydantic.video_input import VideoInput
from aana.models.pydantic.whisper_params import WhisperParams
from aana.utils.video import download_video, extract_audio
from aana.models.core.video import Video
from aana.models.core.audio import Audio


class Transcription(TypedDict):
    transcription: str


async def transcribe_video(
    video_input: VideoInput, whisper_params: WhisperParams
) -> Transcription:
    video: Video = download_video(video_input=video_input)

    audio: Audio = extract_audio(video=video)

    whisper_output = await get_deployment(
        "whisper_deployment_medium"
    ).transcribe.remote(audio=audio, params=whisper_params)

    transcription = whisper_output["transcription"]

    return {
        "transcription": transcription.text,
    }

In [9]:
output = await transcribe_video(
    video_input=VideoInput(url="https://www.youtube.com/watch?v=9bZkp7q19f0"),
    whisper_params=WhisperParams(),
)
output

[youtube] Extracting URL: https://www.youtube.com/watch?v=9bZkp7q19f0
[youtube] 9bZkp7q19f0: Downloading webpage
[youtube] 9bZkp7q19f0: Downloading ios player API JSON
[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


2024-04-10 14:39:04,214	INFO router.py:959 -- Using router <class 'ray.serve._private.router.PowerOfTwoChoicesReplicaScheduler'>.


2024-04-10 14:39:04,229	INFO router.py:496 -- Got updated replicas for deployment 'WhisperDeployment' in application 'whisper_deployment_medium': {'whisper_deployment_medium#WhisperDeployment#xpUONH'}.
2024-04-10 14:39:14,399	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': set().
2024-04-10 14:39:25,138	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': {'RequestHandler#RequestHandler#BcyLgF'}.
2024-04-10 14:41:41,138	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': set().
2024-04-10 14:41:53,277	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': {'RequestHandler#RequestHandler#PvfOLZ'}.
2024-04-10 14:43:03,910	INFO router.py:496 -- Got updated replicas for deployment 'RequestHandler' in application 'RequestHandler': set().
2024-04-10 14:43:15,122	INFO rout

{'transcription': " 오빤 강남 스타일 낮에는 따사로운 인간적인 여자 커피 한 잔에 여유를 아는 품격 있는 여자 밤이 오면 심장이 뜨거워지는 여자 그런 반전 있는 여자 너는 서너에 낮에는 너만큼 따사로운 그런 서너에 커피 씻기도 전에 원샷 때리는 서너에 밤이 오면 심장이 터져버리는 서너에 그런 서너에 아름다워 사랑스러워 그래 너, 그래 바로 너 아름다워 사랑스러워 그래 너, 그래 바로 너 지금부터 갈 때까지 가볼까? Go! 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 정숙해 보이지만 놀 땐 노는 여자 이대다 싶으면 묶었던 머리 푸는 여자 가렸지만 웬만한 노출보다 야한 여자 그런 감각적인 여자 너는 서너에 점잖아 보이지만 놀 땐 노는 서너에 그때가 되면 완전 미쳐버리는 서너에 근육보다 사상이 올통볼통한 서너에 그런 서너에 아름다워 사랑스러워 그래 너, 그래 바로 너 아름다워 사랑스러워 그래 너, 그래 바로 너 지금부터 갈 때까지 가볼까? Go! 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 뛰는 놈 그 위에 나는 놈 베이베 베이베 나는 뭘 좀 아는 놈 뛰는 놈 그 위에 나는 놈 베이베 베이베 나는 뭘 좀 아는 놈 You know what I'm saying 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 오빤 강남 스타일"}

In [10]:
aana_sdk.register_endpoint(
    name="transcribe_video",
    path="/transcribe_video",
    summary="Transcribe video",
    func=transcribe_video,
)

In [11]:
aana_sdk.start_request_handler()

DeploymentHandle(deployment='RequestHandler')

In [12]:
import requests, json

data = {"video_input": {"url": "https://www.youtube.com/watch?v=9bZkp7q19f0"}}
requests.post(
    "http://127.0.0.1:8000/transcribe_video", data={"body": json.dumps(data)}
).json()

{'transcription': " 오빤 강남 스타일 낮에는 따사로운 인간적인 여자 커피 한 잔에 여유를 아는 품격 있는 여자 밤이 오면 심장이 뜨거워지는 여자 그런 반전 있는 여자 너는 서너에 낮에는 너만큼 따사로운 그런 서너에 커피 씻기도 전에 원샷 때리는 서너에 밤이 오면 심장이 터져버리는 서너에 그런 서너에 아름다워 사랑스러워 그래 너, 그래 바로 너 아름다워 사랑스러워 그래 너, 그래 바로 너 지금부터 갈 때까지 가볼까? Go! 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 정숙해 보이지만 놀 땐 노는 여자 이대다 싶으면 묶었던 머리 푸는 여자 가렸지만 웬만한 노출보다 야한 여자 그런 감각적인 여자 너는 서너에 점잖아 보이지만 놀 땐 노는 서너에 그때가 되면 완전 미쳐버리는 서너에 근육보다 사상이 올통볼통한 서너에 그런 서너에 아름다워 사랑스러워 그래 너, 그래 바로 너 아름다워 사랑스러워 그래 너, 그래 바로 너 지금부터 갈 때까지 가볼까? Go! 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 뛰는 놈 그 위에 나는 놈 베이베 베이베 나는 뭘 좀 아는 놈 뛰는 놈 그 위에 나는 놈 베이베 베이베 나는 뭘 좀 아는 놈 You know what I'm saying 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 에이 섹시 레이더 오빤 강남 스타일 오빤 강남 스타일"}

In [15]:
from typing import AsyncGenerator, TypedDict
from aana.models.pydantic.asr_output import AsrTranscription
from aana.models.pydantic.video_input import VideoInput
from aana.models.pydantic.whisper_params import WhisperParams
from aana.utils.video import download_video, extract_audio
from aana.models.core.video import Video
from aana.models.core.audio import Audio


class Transcription(TypedDict):
    transcription: str
    full_transcription: str


async def transcribe_video_stream(
    video_input: VideoInput, whisper_params: WhisperParams
) -> AsyncGenerator[Transcription, None]:
    video: Video = download_video(video_input=video_input)
    audio: Audio = extract_audio(video=video)

    transcriptions = []
    async for whisper_output in (
        get_deployment("whisper_deployment_medium")
        .options(stream=True)
        .transcribe_stream.remote(audio=audio, params=whisper_params)
    ):
        transcription = whisper_output["transcription"]
        transcriptions.append(transcription)
        yield {"transcription": transcription.text}

    full_transcription = sum(transcriptions, AsrTranscription())
    yield {"full_transcription": full_transcription.text}

In [16]:
async for chunk in transcribe_video_stream(
    video_input=VideoInput(url="https://www.youtube.com/watch?v=9bZkp7q19f0"),
    whisper_params=WhisperParams(),
):
    print(chunk)

[youtube] Extracting URL: https://www.youtube.com/watch?v=9bZkp7q19f0
[youtube] 9bZkp7q19f0: Downloading webpage


[youtube] 9bZkp7q19f0: Downloading ios player API JSON
[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


[youtube] 9bZkp7q19f0: Downloading android player API JSON


{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 낮에는 따사로운 인간적인 여자'}
{'transcription': ' 커피 한 잔에 여유를 아는 품격 있는 여자'}
{'transcription': ' 밤이 오면 심장이 뜨거워지는 여자'}
{'transcription': ' 그런 반전 있는 여자'}
{'transcription': ' 너는 서너에'}
{'transcription': ' 낮에는 너만큼 따사로운 그런 서너에'}
{'transcription': ' 커피 씻기도 전에 원샷 때리는 서너에'}
{'transcription': ' 밤이 오면 심장이 터져버리는 서너에'}
{'transcription': ' 그런 서너에'}
{'transcription': ' 아름다워 사랑스러워'}
{'transcription': ' 그래 너, 그래 바로 너'}
{'transcription': ' 아름다워 사랑스러워'}
{'transcription': ' 그래 너, 그래 바로 너'}
{'transcription': ' 지금부터 갈 때까지 가볼까?'}
{'transcription': ' Go!'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 에이 섹시 레이더'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 에이 섹시 레이더'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 정숙해 보이지만 놀 땐 노는 여자'}
{'transcription': ' 이대다 싶으면 묶었던 머리 푸는 여자'}
{'transcription': ' 가렸지만 웬만한 노출보다 야한 여자'}
{'transcription': ' 그런 감각적인 여자'}
{'transcription': ' 너는 서너에'}
{'transcription': ' 점잖아 보이지만 놀 땐 노는 서너에'}
{'transcription': ' 그때가 되면 완전 미

In [23]:
aana_sdk.register_endpoint(
    name="transcribe_video_stream",
    path="/transcribe_video_stream",
    summary="Transcribe video as generator",
    func=transcribe_video_stream,
)

In [24]:
aana_sdk.start_request_handler()

DeploymentHandle(deployment='RequestHandler')

In [30]:
import requests, json

data = {"video_input": {"url": "https://www.youtube.com/watch?v=9bZkp7q19f0"}}
response = requests.post(
    "http://127.0.0.1:8000/transcribe_video_stream",
    data={"body": json.dumps(data)},
    stream=True,
)
for chunk in response.iter_content(chunk_size=None):
    print(json.loads(chunk))

(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] Extracting URL: https://www.youtube.com/watch?v=9bZkp7q19f0
(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading webpage
(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading ios player API JSON
(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading android player API JSON


(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] YouTube said: ERROR - Precondition check failed.
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] HTTP Error 400: Bad Request. Retrying (1/3)...
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] YouTube said: ERROR - Precondition check failed.
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] HTTP Error 400: Bad Request. Retrying (2/3)...
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] YouTube said: ERROR - Precondition check failed.
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] HTTP Error 400: Bad Request. Retrying (3/3)...
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] YouTube said: ERROR - Precondition check failed.
(ServeReplica:RequestHandler:RequestHandler pid=108862) WARNING: [youtube] Unable to download API page: HTTP Error 400: Bad Request (cause

(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading android player API JSON
(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading android player API JSON
(ServeReplica:RequestHandler:RequestHandler pid=108862) [youtube] 9bZkp7q19f0: Downloading android player API JSON


{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 낮에는 따사로운 인간적인 여자'}
{'transcription': ' 커피 한 잔에 여유를 아는 품격 있는 여자'}
{'transcription': ' 밤이 오면 심장이 뜨거워지는 여자'}
{'transcription': ' 그런 반전 있는 여자'}
{'transcription': ' 너는 서너에'}
{'transcription': ' 낮에는 너만큼 따사로운 그런 서너에'}
{'transcription': ' 커피 씻기도 전에 원샷 때리는 서너에'}
{'transcription': ' 밤이 오면 심장이 터져버리는 서너에'}
{'transcription': ' 그런 서너에'}
{'transcription': ' 아름다워 사랑스러워'}
{'transcription': ' 그래 너, 그래 바로 너'}
{'transcription': ' 아름다워 사랑스러워'}
{'transcription': ' 그래 너, 그래 바로 너'}
{'transcription': ' 지금부터 갈 때까지 가볼까?'}
{'transcription': ' Go!'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 에이 섹시 레이더'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 에이 섹시 레이더'}
{'transcription': ' 오빤 강남 스타일'}
{'transcription': ' 정숙해 보이지만 놀 땐 노는 여자'}
{'transcription': ' 이대다 싶으면 묶었던 머리 푸는 여자'}
{'transcription': ' 가렸지만 웬만한 노출보다 야한 여자'}
{'transcription': ' 그런 감각적인 여자'}
{'transcription': ' 너는 서너에'}
{'transcription': ' 점잖아 보이지만 놀 땐 노는 서너에'}
{'transcription': ' 그때가 되면 완전 미